In [6]:

import warnings
warnings.filterwarnings("ignore")

In [7]:
from environments.live_environments import BaseLiveTradingEnv
from neuralforecast.core import NeuralForecast
from Keys import *
import pickle
import numpy as np
from utils import pearl_utils,utils,discord_utils
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment
import boto3
from discord import SyncWebhook
from utils import discord_utils
import requests
import matplotlib.pyplot as plt
import pandas as pd
from utils.pandas_reward_functions import log_reward_function,diff_reward_function

In [8]:
channel_url='https://discord.com/api/webhooks/986694946381783102/FOA7nG9ShDcXY95-c3XEKV-Fdek66L9xfbQoKuEuFQkK2P4aFWaZ_fKmzw00j8Oj8Woj'
message='Testing'

In [9]:
from configs import fx_defaults as defaults


In [10]:
forecast_model=NeuralForecast.load(defaults.forecasting_model_path)

# forecast_model=NeuralForecast.load('MultiHeadForecastingModel/')

Seed set to 12
Seed set to 5


In [11]:
mod=forecast_model.models[0]
dir(mod)

['CHECKPOINT_HYPER_PARAMS_KEY',
 'CHECKPOINT_HYPER_PARAMS_NAME',
 'CHECKPOINT_HYPER_PARAMS_TYPE',
 'EXOGENOUS_FUTR',
 'EXOGENOUS_HIST',
 'EXOGENOUS_STAT',
 'SAMPLING_TYPE',
 'T_destination',
 '_LightningModule__check_allowed',
 '_LightningModule__check_not_nested',
 '_LightningModule__to_tensor',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_apply_batch_transfer_handler',
 '_automatic_optimization',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_batch_hook',
 '_call_impl',
 '_check_exog',
 '_compiled_call_impl',
 '_compiler_c

In [12]:
forecast_model.scalers_

{'y': <coreforecast.scalers.LocalRobustScaler at 0x103b47ca0>,
 'feature_hour_of_day': <coreforecast.scalers.LocalRobustScaler at 0x168a931c0>,
 'feature_day_of_week': <coreforecast.scalers.LocalRobustScaler at 0x168a93f40>,
 'feature_day_of_month': <coreforecast.scalers.LocalRobustScaler at 0x168a93fd0>,
 'feature_day_of_year': <coreforecast.scalers.LocalRobustScaler at 0x168a92f20>,
 'feature_BBANDS_BB_UPPER': <coreforecast.scalers.LocalRobustScaler at 0x168a93ca0>,
 'feature_BBANDS_BB_MIDDLE': <coreforecast.scalers.LocalRobustScaler at 0x168a93f70>,
 'feature_BBANDS_BB_LOWER': <coreforecast.scalers.LocalRobustScaler at 0x168a93eb0>,
 'feature_MACD_MACD': <coreforecast.scalers.LocalRobustScaler at 0x16896dae0>,
 'feature_MACD_SIGNAL': <coreforecast.scalers.LocalRobustScaler at 0x168bf2050>,
 'feature_14_period_RSI': <coreforecast.scalers.LocalRobustScaler at 0x168bf17e0>,
 'feature_14_period_STOCH_%K': <coreforecast.scalers.LocalRobustScaler at 0x168bf2110>}

In [13]:
base_asset=defaults.base_asset
quote_asset=defaults.quote_asset   
test_net=True
time_frame=defaults.time_frame
product_type=defaults.product_type
exchange=defaults.exchange
trade_target=defaults.target_pair
trade_target

'USDJPY'

In [14]:
live_env=BaseLiveTradingEnv(
            api_key=oanda_api_key,
            account_id=oanda_account_id,
            paper=test_net,
            symbol=trade_target,
            time_frame=time_frame,
            product_type=product_type,
            positions=[-1,1],
            history_path='Trade_history/trade.db',
            exchange=exchange,
            forecast_model=forecast_model,
            reward_function=diff_reward_function,
            
            discord_webhook=discord_forex_webhook

            )

Connected to oanda client
Getting data


100%|██████████| 78/78 [00:00<00:00, 11944.78it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
live_env.client.symbol,live_env.client.base_asset,live_env.client.quote_asset

In [ ]:
live_env.reset()
# live_env.step(0)

In [ ]:
# live_env.client._position_frame.loc[live_env.symbol,'balance']
history

In [ ]:
live_env.client._account


In [ ]:
plot_df=live_env.prepare_plot_df()
history=live_env.load_history()

plot_df['ds']=pd.to_datetime(plot_df['ds']).dt.tz_localize('US/Pacific')
plot_df['ds'].reset_index(drop=True)
history['date']=pd.to_datetime([pd.Timestamp(d) for d in history['date']]).tz_convert('US/Pacific')
history['date']
plot_df.iloc[-1]['ds']


In [ ]:
history.iloc[0]['date']

In [104]:
history=history[history['date']<=plot_df.iloc[-1]['ds']].copy()
history=history[history['date']>=plot_df.iloc[0]['ds']].copy()
plot_df=plot_df[plot_df['ds']>history.iloc[0]['date']]

In [105]:

change_history=history[history['Current_position']!=history['Trade_to']]
change_history=change_history[['date','Trade_to','Current_position','price','portfolio_valuation']]
change_history['Trade_to']=change_history['Trade_to'].astype(int)
change_history['price']=change_history['price'].astype(float)
change_history['portfolio_valuation']=change_history['portfolio_valuation'].astype(float)

In [ ]:

buys=change_history[change_history['Trade_to']==1].copy()
sells=change_history[change_history['Trade_to']<1].copy()
sells

In [110]:
plot_df['close']=plot_df['close'].astype(float)

In [ ]:
plot_df['close']

In [ ]:
fig,axes=plt.subplots(sharex=False, sharey=False, figsize=(10, 5))
time=pd.Timestamp(plot_df['ds'].values[-1]).strftime('%m-%d-%Y %I:%M%p')
plot_df['ds']=pd.to_datetime(plot_df['ds'])
axes.plot(plot_df['ds'], plot_df['close'], label='Close Price')


# axes.scatter(buys['date'],buys['price'],color='green',label='Buys',marker='^')
# axes.scatter(sells['date'],sells['price'],color='red',label='Sells',marker='v')
# plot_df['ds'].values[-1]
# axes.legend()


In [ ]:
live_env.raw_df

In [ ]:
model_name=defaults.model_name
agent_path=f'Agent/pearl_{model_name}_model.pkl'
agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
                                        study_name=f'pearl-{model_name}-hp-search',
                                        action_space_dim=2,
                                            observation_space_dim=30,)
agent=pearl_utils.load_agent_weights(agent,weight_path=agent_path)

In [ ]:
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment

In [ ]:
live_pearl_env=GymEnvironment(live_env)

In [ ]:
observation,action_space=live_pearl_env.reset()

In [ ]:
agent.observe(observation, action_space)

In [ ]:
action=agent.act(exploit=True)

# action=
action

In [ ]:
live_env.client.account()

In [ ]:
action_result=live_pearl_env.step(int(action))


In [ ]:
len(live_env.client.trade_client.get_fills()['fills'])

In [ ]:
action_result.observation

In [ ]:
df=live_env.df
import pandas as pd

In [ ]:
df['ds_pst']=pd.to_datetime(df['ds'],utc=False)
df[['ds','ds_pst']]

In [ ]:
live_env.client.get_trade_rules()